In [2]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import statsmodels.api as sm
import Fish_data

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [24]:
f = Fish_data.Fish()

In [40]:
data = f.get_fish_data()
data = data.set_index("FishID")
data.head()

,Date,TREND,Gear,Species,Sex,Length,Mass,Ktl,Relative weight,Maturity,...,parasite,misc 1 text,misc 2 num,misc 3 text,misc 4 num,Site,KFL,Day,Month,Year
FishID,,,,,,,,,,,,,,,,,,,,,
19710001,1971-03-09 00:00:00,1,GN,LMB,NaN,263,250.0,1.37,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,RN,0.0,9.0,3.0,1971.0
19710002,1971-03-09 00:00:00,1,GN,LMB,NaN,348,700.0,1.66,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,RN,0.0,9.0,3.0,1971.0
19710003,1971-03-09 00:00:00,1,GN,LMB,NaN,332,555.0,1.51,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,RN,NaN,9.0,3.0,1971.0
19710004,1971-03-09 00:00:00,1,GN,LMB,NaN,350,720.0,1.67,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,RN,NaN,9.0,3.0,1971.0
19710005,1971-03-09 00:00:00,1,GN,LMB,NaN,300,455.0,1.68,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,RN,NaN,9.0,3.0,1971.0


In [41]:
#fix weights
weight = data[['Relative weight']]
weight = weight.fillna(0)
data['new_weight'] = weight

In [42]:
#fix dates
from datetime import datetime
datetime_object = datetime.strptime('1700-3-01', '%Y-%m-%d')
data['new_date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d %H:%M', errors='coerce')
data = data[data['new_date'] > datetime_object]

In [43]:
#select for fish with data
# specieswithweights = data.loc[data['new_weight']!=0]
STBspecies = data.loc[(data['Species'] == "STB") | (data['Species'] == "stb")]

SMBspecies = data.loc[(data['Species'] == "SMB") | (data['Species'] == "smb")]


In [44]:
STBspecies.head()

,Date,TREND,Gear,Species,Sex,Length,Mass,Ktl,Relative weight,Maturity,...,misc 2 num,misc 3 text,misc 4 num,Site,KFL,Day,Month,Year,new_weight,new_date
FishID,,,,,,,,,,,,,,,,,,,,,
19770087,1977-03-24 00:00:00,1,GN,STB,NaN,421,900.0,1.20,NaN,NaN,...,NaN,NaN,NaN,PB,1.0,24.0,3.0,1977.0,0.0,1977-03-24
19770088,1977-03-24 00:00:00,1,GN,STB,NaN,463,1180.0,1.18,96.0,NaN,...,NaN,NaN,NaN,PB,1.0,24.0,3.0,1977.0,96.0,1977-03-24
19770089,1977-03-23 00:00:00,1,GN,STB,NaN,320,500.0,1.52,123.0,NaN,...,NaN,NaN,NaN,RN,2.0,23.0,3.0,1977.0,123.0,1977-03-23
19770090,1977-03-23 00:00:00,1,GN,STB,NaN,310,525.0,1.76,142.0,NaN,...,NaN,NaN,NaN,RN,2.0,23.0,3.0,1977.0,142.0,1977-03-23
19770091,1977-03-23 00:00:00,1,GN,STB,NaN,375,900.0,1.70,137.0,NaN,...,NaN,NaN,NaN,RN,2.0,23.0,3.0,1977.0,137.0,1977-03-23


In [55]:
# define the data/predictors as the pre-set feature names  
df = pd.DataFrame(STBspecies["Length"])

# Put the target (housing value -- MEDV) in another DataFrame
target = pd.DataFrame(STBspecies['new_weight'])

In [49]:
df.head()
target["new_weight"].unique()

array([  0.,  96., 123., 142., 137., 141., 143., 145., 125., 135., 122.,
       127., 154., 110., 128., 133., 147., 109.,  88., 101.,  92.,  93.,
        90.,  94.,  84.,  78.,  86.,  82.,  80.,  85.,  87.,  91.,  98.,
        81.,  89.,  83.,  53.,  97.,  72., 103., 120., 111., 100., 104.,
        19., 114., 107., 118.,  20.,  99., 115., 113., 108., 398., 102.,
       117., 105., 126., 139., 130.,  51., 131., 112., 124.,   9.,  70.,
        63.,  62.,  67.,  74.,  71.,  59., 119., 106.,  77.,  95.,  73.,
       129.,  69.,  76.,  64.,  38.,  40.,  79.,  75., 394.,  43.,  39.,
       158., 121.,  42., 116.,  66.,  65.,  57.,  68.,  61., 162., 180.,
       169.,  58., 252., 562.,  54.,  60.,  50.,  41.,  55.,  45.,  56.,
        24.,  32., 134., 148., 136.,  47.,  35.,  34.,  52.,  49.,  18.,
        46., 291.,  48.,  44.,  36., 379.,  37., 324., 242., 173., 155.,
       246., 153.,  25., 152., 349., 359., 400., 302., 321., 335., 366.,
       183., 156., 164.,  21., 140., 144., 159.,  3

In [50]:
import statsmodels.api as sm

In [56]:
X = df["Length"]
y = target["new_weight"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             new_weight   R-squared (uncentered):                   0.797
Model:                            OLS   Adj. R-squared (uncentered):              0.797
Method:                 Least Squares   F-statistic:                          6.444e+04
Date:                Thu, 20 Feb 2020   Prob (F-statistic):                        0.00
Time:                        10:08:08   Log-Likelihood:                         -82199.
No. Observations:               16454   AIC:                                  1.644e+05
Df Residuals:                   16453   BIC:                                  1.644e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Length         0.1858      0.001    253.849      0.000       0.184       0.187
==============================================================================
Omnibus:                    11500.269   Durbin-Watson:                   0.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1202964.206
Skew:                           2.553   Prob(JB):                         0.00
Kurtosis:                      44.576   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""